In [ ]:
import multiprocessing
import Queue

import numpy as np
import logging
import csv
import datetime
import time
import random
import sys


# constant for logger
CURRENT_TIMESTAMP = datetime.datetime.fromtimestamp(time.time()).strftime('%Y_%m_%d_%H_%M_%S')
RESULT_OUTPUT = 'run_rnn_mt_result_' + CURRENT_TIMESTAMP + '.log'
LOGGER_FORMAT_HEADER = '%(asctime)s, %(levelname)s, %(message)s'
CUTOFF_LINE = '------------------------------------------------------------------------'
IS_ENABLE_FILE_LOGGING = True

# general constant
SPLIT_RANDOM_STATE = 42
TEST_SIZE = 0.33

# constant for mt
ENABLE_SIMPLE_RUN = False
THREAD_COUNT = 10
SIMPLE_LEARNING_LEN = 1
SIMPLE_BATCH_LEN = 1
SIMPLE_HIDDEN_LEN = 1


# constant for rnn training
# learning_rate = 0.001
LEARNING_RATE_RANGE = [0.001, 0.005, 0.01]
training_steps = 40000 # TODO: change the steps to 10000 for better result
# batch_size = 128
BATCH_SIZE_RANGE = [64, 128, 256, 512]
display_step = 100

# constant rnn network parameters
num_input = 3 # we only read one set of yaw pitch row
timesteps = 100  # timesteps - we have 100 data point for each char
# num_hidden = 128  # hidden layer num of features
NUM_HIDDEN_RANGE = [64, 128, 256, 512]
num_classes = 5  # number of data class - using a/b/c/d/e

# raw data file names
DATA_SET_A = 'run_letter_a_format.csv'
DATA_SET_B = 'run_letter_b_format.csv'
DATA_SET_C = 'run_letter_c_format.csv'
DATA_SET_D = 'run_letter_d_format.csv'
DATA_SET_E = 'run_letter_e_format.csv'


# bookkeeping logic for setup logger and random sample generator
LOGGER = logging.getLogger('cogs181_runtime')
LOGGER.setLevel(logging.DEBUG)
formatter = logging.Formatter(LOGGER_FORMAT_HEADER)

# setup file logging if user enable
if IS_ENABLE_FILE_LOGGING:
    fh = logging.FileHandler(RESULT_OUTPUT)
    fh.setLevel(logging.DEBUG)
    fh.setFormatter(formatter)
    LOGGER.addHandler(fh)

ch = logging.StreamHandler()
ch.setLevel(logging.DEBUG)
ch.setFormatter(formatter)
LOGGER.addHandler(ch)

random.seed(SPLIT_RANDOM_STATE)


def read_format_input(read_file_name):
    with open(read_file_name, 'r') as f:
        reader = csv.reader(f)
        raw_data_list = list(reader)

    return raw_data_list


def render_raw_data():
    from sklearn.model_selection import train_test_split
    raw_a_x = np.array(read_format_input(DATA_SET_A)).astype(None)
    raw_b_x = np.array(read_format_input(DATA_SET_B)).astype(None)
    raw_c_x = np.array(read_format_input(DATA_SET_C)).astype(None)
    raw_d_x = np.array(read_format_input(DATA_SET_D)).astype(None)
    raw_e_x = np.array(read_format_input(DATA_SET_E)).astype(None)
    raw_x = np.concatenate((raw_a_x, raw_b_x, raw_c_x, raw_d_x, raw_e_x), axis=0)

    raw_a_y = np.array([[1, 0, 0, 0, 0]] * len(raw_a_x)).astype(None)
    raw_b_y = np.array([[0, 1, 0, 0, 0]] * len(raw_b_x)).astype(None)
    raw_c_y = np.array([[0, 0, 1, 0, 0]] * len(raw_c_x)).astype(None)
    raw_d_y = np.array([[0, 0, 0, 1, 0]] * len(raw_d_x)).astype(None)
    raw_e_y = np.array([[0, 0, 0, 0, 1]] * len(raw_e_x)).astype(None)
    raw_y = np.concatenate((raw_a_y, raw_b_y, raw_c_y, raw_d_y, raw_e_y), axis=0)

    train_x, test_x, train_y, test_y = \
        train_test_split(raw_x, raw_y, test_size=TEST_SIZE, random_state=SPLIT_RANDOM_STATE)

    return train_x, train_y, test_x, test_y


# TODO: make render_batch generate next set of unique batch instead of repeat some same index
def render_batch(batch_size, x_data, y_data):
    if len(x_data) != len(y_data):
        sys.exit("Error: cannot render batch with different len of x and y.")

    batch_index = random.sample(range(0, len(x_data)), batch_size)
    render_x = []
    render_y = []

    for index in batch_index:
        render_x.append(x_data[index])
        render_y.append(y_data[index])

    render_x = np.array(render_x).astype(None)
    render_y = np.array(render_y).astype(None)
    return render_x, render_y


def rnn_training_engine_worker(exp_id, train_x, train_y, test_x, test_y, layer_index, learning_index, batch_index):
    def get_param(num_hidden, ):
        import tensorflow as tf
        # tf Graph input
        X = tf.placeholder("float", [None, timesteps, num_input])
        Y = tf.placeholder("float", [None, num_classes])

        weights = {
            'out': tf.Variable(tf.random_normal([num_hidden, num_classes]))
        }
        biases = {
            'out': tf.Variable(tf.random_normal([num_classes]))
        }

        return X, Y, weights, biases

    def rnn_nodes(x, weights, biases, num_hidden):
        import tensorflow as tf
        from tensorflow.contrib import rnn
        with tf.variable_scope('scope', reuse=tf.AUTO_REUSE):

            def lstm_cell():
                return rnn.BasicLSTMCell(num_hidden, forget_bias=1.0, reuse=tf.get_variable_scope().reuse)

            # Prepare data shape to match `rnn` function requirements
            # Current data input shape: (batch_size, timesteps, n_input)
            # Required shape: 'timesteps' tensors list of shape (batch_size, n_input)

            # unstack to get a list of 'timesteps' tensors of shape (batch_size, n_input)
            x = tf.unstack(x, timesteps, 1)

            # get lstm cell output
            outputs, states = rnn.static_rnn([lstm_cell()][0], x, dtype=tf.float32)

            # linear activation, using rnn inner loop last output
            return tf.matmul(outputs[-1], weights['out']) + biases['out']

    LOGGER.debug("exp id, " + str(exp_id) + ", Start RNN worker")

    X, Y, weights, biases = get_param(NUM_HIDDEN_RANGE[layer_index], )
    import tensorflow as tf
    logits = rnn_nodes(X, weights, biases, NUM_HIDDEN_RANGE[layer_index])
    prediction = tf.nn.softmax(logits)

    # define loss and optimizer
    loss_op = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits(logits=logits, labels=Y))
    optimizer = tf.train.GradientDescentOptimizer(learning_rate=LEARNING_RATE_RANGE[learning_index])
    train_op = optimizer.minimize(loss_op)

    # evaluate model (with test logits, for dropout to be disabled)
    correct_pred = tf.equal(tf.argmax(prediction, 1), tf.argmax(Y, 1))
    accuracy = tf.reduce_mean(tf.cast(correct_pred, tf.float32))

    # assign vars to default value
    init = tf.global_variables_initializer()

    # start tf training
    config = tf.ConfigProto()
    config.gpu_options.allow_growth = True

    with tf.Session(config=config) as sess:

        # run the initializer
        sess.run(init)

        for step in range(1, training_steps + 1):
            batch_x, batch_y = render_batch(BATCH_SIZE_RANGE[batch_index], train_x, train_y)
            # reshape data to get 100 seq of 3 elements (y,p,r)
            batch_x = batch_x.reshape((BATCH_SIZE_RANGE[batch_index], timesteps, num_input))
            # run optimization operation by using backprop
            sess.run(train_op, feed_dict={X: batch_x, Y: batch_y})
            if step % display_step == 0 or step == 1:

                # calculate batch loss and accuracy
                loss, acc = sess.run([loss_op, accuracy], feed_dict={X: batch_x, Y: batch_y})
                training_step_log = ", Step, " + str(step) + ", Minibatch Loss, " + "{:.5f}".format(loss) + ", Training Accuracy, " + "{:.5f}".format(acc)
                LOGGER.debug("exp id, " + str(exp_id) + training_step_log)

        LOGGER.debug("exp id, " + str(exp_id) + ", Optimization Finished!")

        # calculate accuracy on testing set
        test_data = test_x.reshape((-1, timesteps, num_input))
        test_label = test_y
        test_acc = sess.run(accuracy, feed_dict={X: test_data, Y: test_label})
        test_log = ", Testing Accuracy, " + str(test_acc)
        LOGGER.debug("exp id, " + str(exp_id) + test_log)

        train_data = train_x.reshape((-1, timesteps, num_input))
        train_label = train_y
        train_acc = sess.run(accuracy, feed_dict={X: train_data, Y: train_label})
        train_log = ", Training Accuracy, " + str(train_acc)

        LOGGER.debug("exp id, " + str(exp_id) + train_log)
        return "exp id, " + str(exp_id) + (train_log + test_log)


def rnn_training_master(train_x, train_y, test_x, test_y, is_simple_run=False):
    import tensorflow as tf
    rnn_results = []
    exp_id = 0

    num_layer_length = len(NUM_HIDDEN_RANGE)
    learning_length = len(LEARNING_RATE_RANGE)
    batch_length = len(BATCH_SIZE_RANGE)

    if is_simple_run:
        num_layer_length = SIMPLE_HIDDEN_LEN
        learning_length = SIMPLE_LEARNING_LEN
        batch_length = SIMPLE_BATCH_LEN

    exp_params_queue = Queue.Queue()
    for layer_index in range(num_layer_length):
        for learning_index in range(learning_length):
            for batch_index in range(batch_length):
                exp_params_queue.put((layer_index, learning_index, batch_index))

    tf.reset_default_graph()

    while not exp_params_queue.empty():
        flush_mt_counter = THREAD_COUNT
        rnn_results_temp = []
        rnn_pool = multiprocessing.Pool(processes=THREAD_COUNT)

        # while flush_mt_counter > 1 and not exp_params_queue.empty():
        while flush_mt_counter > 0 and not exp_params_queue.empty():
            flush_mt_counter -= 1

            current_exp_params = exp_params_queue.get()
            sample_result = rnn_pool.apply_async(rnn_training_engine_worker, (exp_id, train_x, train_y, test_x, test_y, current_exp_params[0], current_exp_params[1], current_exp_params[2],))
            exp_id += 1
            rnn_results_temp.append(sample_result)

        rnn_pool.close()
        rnn_pool.join()

        for sample_result in rnn_results_temp:
            rnn_results.append(sample_result)

        LOGGER.debug("Flush RNN training master\n" + CUTOFF_LINE)

    rnn_results = transform_apply_result(rnn_results)
    return rnn_results


def transform_apply_result(input_list):
    result_list = []

    for i in range(len(input_list)):
        result_list.append(input_list[i].get())

    return result_list


def main():
    LOGGER.debug("Start reading formatted input")
    train_x, train_y, test_x, test_y = render_raw_data()
    LOGGER.debug("End reading formatted input\n" + CUTOFF_LINE)

    LOGGER.debug("Start RNN training master")
    master_res = rnn_training_master(train_x, train_y, test_x, test_y, is_simple_run=ENABLE_SIMPLE_RUN)
    LOGGER.debug("End RNN training master\n" + CUTOFF_LINE)

    LOGGER.debug("Start Exp summary report")
    for res in master_res:
        LOGGER.debug(res)

    LOGGER.debug("End Exp summary report\n" + CUTOFF_LINE)


if __name__ == '__main__':
    main()


2017-12-15 22:54:04,131, DEBUG, Start reading formatted input
2017-12-15 22:54:05,148, DEBUG, End reading formatted input
------------------------------------------------------------------------
2017-12-15 22:54:05,149, DEBUG, Start RNN training master
2017-12-15 22:54:06,681, DEBUG, exp id, 0, Start RNN worker
2017-12-15 22:54:06,709, DEBUG, exp id, 1, Start RNN worker
2017-12-15 22:54:06,745, DEBUG, exp id, 2, Start RNN worker
2017-12-15 22:54:06,777, DEBUG, exp id, 3, Start RNN worker
2017-12-15 22:54:06,928, DEBUG, exp id, 4, Start RNN worker
2017-12-15 22:54:07,032, DEBUG, exp id, 5, Start RNN worker
2017-12-15 22:54:07,150, DEBUG, exp id, 6, Start RNN worker
2017-12-15 22:54:07,340, DEBUG, exp id, 7, Start RNN worker
2017-12-15 22:54:07,540, DEBUG, exp id, 8, Start RNN worker
2017-12-15 22:54:07,742, DEBUG, exp id, 9, Start RNN worker
2017-12-15 22:55:39,941, DEBUG, exp id, 6, Step, 1, Minibatch Loss, 3.70231, Training Accuracy, 0.19531
2017-12-15 22:55:42,327, DEBUG, exp id, 2, 

2017-12-15 23:00:36,428, DEBUG, exp id, 5, Step, 700, Minibatch Loss, 0.64521, Training Accuracy, 0.82031
2017-12-15 23:00:36,527, DEBUG, exp id, 3, Step, 600, Minibatch Loss, 0.75431, Training Accuracy, 0.72461
2017-12-15 23:00:37,325, DEBUG, exp id, 7, Step, 600, Minibatch Loss, 0.39725, Training Accuracy, 0.87109
2017-12-15 23:00:38,239, DEBUG, exp id, 4, Step, 700, Minibatch Loss, 0.55200, Training Accuracy, 0.81250
2017-12-15 23:00:55,644, DEBUG, exp id, 1, Step, 700, Minibatch Loss, 0.77931, Training Accuracy, 0.69531
2017-12-15 23:01:02,137, DEBUG, exp id, 0, Step, 700, Minibatch Loss, 1.08252, Training Accuracy, 0.67188
2017-12-15 23:01:07,037, DEBUG, exp id, 9, Step, 700, Minibatch Loss, 0.37439, Training Accuracy, 0.89062
2017-12-15 23:01:07,128, DEBUG, exp id, 8, Step, 700, Minibatch Loss, 0.52305, Training Accuracy, 0.81250
2017-12-15 23:01:12,730, DEBUG, exp id, 6, Step, 800, Minibatch Loss, 0.52448, Training Accuracy, 0.83984
2017-12-15 23:01:18,334, DEBUG, exp id, 5, Ste

2017-12-15 23:06:10,226, DEBUG, exp id, 6, Step, 1500, Minibatch Loss, 0.51785, Training Accuracy, 0.80859
2017-12-15 23:06:12,527, DEBUG, exp id, 5, Step, 1500, Minibatch Loss, 0.44583, Training Accuracy, 0.87500
2017-12-15 23:06:15,631, DEBUG, exp id, 2, Step, 1500, Minibatch Loss, 0.63264, Training Accuracy, 0.77344
2017-12-15 23:06:15,742, DEBUG, exp id, 9, Step, 1400, Minibatch Loss, 0.42432, Training Accuracy, 0.89062
2017-12-15 23:06:17,526, DEBUG, exp id, 8, Step, 1400, Minibatch Loss, 0.36260, Training Accuracy, 0.95312
2017-12-15 23:06:35,639, DEBUG, exp id, 1, Step, 1500, Minibatch Loss, 0.57267, Training Accuracy, 0.79688
2017-12-15 23:06:46,927, DEBUG, exp id, 3, Step, 1400, Minibatch Loss, 0.63149, Training Accuracy, 0.76758
2017-12-15 23:06:47,732, DEBUG, exp id, 7, Step, 1400, Minibatch Loss, 0.39250, Training Accuracy, 0.85938
2017-12-15 23:06:52,025, DEBUG, exp id, 4, Step, 1600, Minibatch Loss, 0.49631, Training Accuracy, 0.79688
2017-12-15 23:06:52,938, DEBUG, exp i

2017-12-15 23:11:47,729, DEBUG, exp id, 5, Step, 2300, Minibatch Loss, 0.39516, Training Accuracy, 0.88281
2017-12-15 23:11:50,528, DEBUG, exp id, 6, Step, 2300, Minibatch Loss, 0.41250, Training Accuracy, 0.86719
2017-12-15 23:11:56,627, DEBUG, exp id, 2, Step, 2300, Minibatch Loss, 0.58607, Training Accuracy, 0.78516
2017-12-15 23:11:57,538, DEBUG, exp id, 0, Step, 2200, Minibatch Loss, 0.59545, Training Accuracy, 0.75000
2017-12-15 23:12:09,331, DEBUG, exp id, 8, Step, 2200, Minibatch Loss, 0.37208, Training Accuracy, 0.89062
2017-12-15 23:12:13,026, DEBUG, exp id, 1, Step, 2300, Minibatch Loss, 0.49425, Training Accuracy, 0.85156
2017-12-15 23:12:14,529, DEBUG, exp id, 3, Step, 2100, Minibatch Loss, 0.53099, Training Accuracy, 0.81250
2017-12-15 23:12:14,729, DEBUG, exp id, 7, Step, 2100, Minibatch Loss, 0.29907, Training Accuracy, 0.91797
2017-12-15 23:12:15,029, DEBUG, exp id, 9, Step, 2200, Minibatch Loss, 0.29485, Training Accuracy, 0.88281
2017-12-15 23:12:22,425, DEBUG, exp i

2017-12-15 23:17:27,828, DEBUG, exp id, 9, Step, 2900, Minibatch Loss, 0.18484, Training Accuracy, 0.95312
2017-12-15 23:17:41,726, DEBUG, exp id, 3, Step, 2800, Minibatch Loss, 0.50211, Training Accuracy, 0.81836
2017-12-15 23:17:43,141, DEBUG, exp id, 7, Step, 2800, Minibatch Loss, 0.23614, Training Accuracy, 0.94141
2017-12-15 23:17:46,730, DEBUG, exp id, 1, Step, 3100, Minibatch Loss, 0.41894, Training Accuracy, 0.85156
2017-12-15 23:17:47,541, DEBUG, exp id, 2, Step, 3100, Minibatch Loss, 0.55902, Training Accuracy, 0.84766
2017-12-15 23:17:49,127, DEBUG, exp id, 0, Step, 3000, Minibatch Loss, 0.30015, Training Accuracy, 0.95312
2017-12-15 23:17:51,736, DEBUG, exp id, 6, Step, 3100, Minibatch Loss, 0.29770, Training Accuracy, 0.91016
2017-12-15 23:17:51,840, DEBUG, exp id, 5, Step, 3100, Minibatch Loss, 0.38477, Training Accuracy, 0.85938
2017-12-15 23:18:01,130, DEBUG, exp id, 4, Step, 3200, Minibatch Loss, 0.49366, Training Accuracy, 0.84375
2017-12-15 23:18:04,326, DEBUG, exp i